<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/live_web_search/duckduckgo_langgraph_search_simple_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# DuckDuckGo + LangGraph: Live Web Search (Jupyter Notebook)

This notebook shows how to wire **DuckDuckGo** search into a minimal **LangGraph** app for live web lookups.
- No API key needed for search (uses the `duckduckgo-search` package).
- Summarization step works without an LLM by default (rule-based), and **optionally** uses OpenAI if you have `OPENAI_API_KEY` set.

> Tested with Python 3.10+.


In [ ]:

# If you're running this on a fresh environment, uncomment and run:
%pip install -q duckduckgo-search langgraph langchain-core langchain-community openai


In [ ]:

from __future__ import annotations

from typing import TypedDict, List, Dict, Optional
from dataclasses import dataclass
from datetime import datetime

# LangGraph core
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# DuckDuckGo search
from duckduckgo_search import DDGS

# Optional OpenAI summarization
import os
OPENAI_AVAILABLE = False
try:
    from openai import OpenAI
    client = OpenAI()
    # A quick capability check; won't throw if no key, but calls will later
    OPENAI_AVAILABLE = True if os.getenv("OPENAI_API_KEY") else False
except Exception:
    OPENAI_AVAILABLE = False

# Small utilities
def _now_iso():
    return datetime.now().isoformat(timespec="seconds")

print("OpenAI available:", OPENAI_AVAILABLE)


In [ ]:

class SearchState(TypedDict, total=False):
    # Input
    query: str
    # Intermediate
    results: List[Dict]
    # Output
    answer_markdown: str


In [ ]:

def search_node(state: SearchState) -> SearchState:
    query = state.get("query", "").strip()
    if not query:
        raise ValueError("Empty query. Provide a non-empty 'query' in the state.")
    
    max_results = 8  # tweak as desired
    results: List[Dict] = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results, safesearch="moderate", region="wt-wt"):
            # r typically has keys like: title, href, body
            results.append({
                "title": r.get("title"),
                "link": r.get("href"),
                "snippet": r.get("body"),
            })
    return {"results": results}


In [ ]:

def _markdown_from_results(results: List[Dict], query: str) -> str:
    if not results:
        return f"**No results found** for: `{query}`"
    
    lines = [f"### Top DuckDuckGo hits for: `{query}` (_{_now_iso()}_)\n"]
    for i, r in enumerate(results, 1):
        title = r.get("title") or "Untitled"
        link = r.get("link") or ""
        snippet = r.get("snippet") or ""
        lines.append(f"{i}. [{title}]({link})\n   - {snippet}")
    return "\n".join(lines)

def _summarize_with_openai(results: List[Dict], query: str) -> Optional[str]:
    if not OPENAI_AVAILABLE:
        return None
    try:
        # Compose a compact prompt
        bullets = "\n".join([f"- {r.get('title')} — {r.get('snippet')}" for r in results[:8]])
        prompt = f"""You are a helpful research assistant.
Summarize the most relevant findings below for the user query: "{query}".
Keep it concise (5-8 bullet points) and avoid redundancy. Include key facts and dates if present.

Findings:
{bullets}
"""
        # Using the OpenAI "Responses" API for simplicity if available.
        resp = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
        )
        # Extract text
        parts = []
        for out in resp.output:
            if out.type == "message":
                for c in out.message.content:
                    if c.type == "text":
                        parts.append(c.text)
        text = "\n".join(parts).strip()
        if text:
            return "### Summary\n" + text
    except Exception as e:
        print("OpenAI summarize failed:", e)
        return None
    return None

def summarize_node(state: SearchState) -> SearchState:
    results = state.get("results", [])
    query = state.get("query", "")
    
    # Always include a markdown list of results.
    md = _markdown_from_results(results, query)
    
    # Optionally append an OpenAI-generated summary if available.
    summary = _summarize_with_openai(results, query)
    if summary:
        md = summary + "\n\n" + md
    
    return {"answer_markdown": md}


In [ ]:

graph = StateGraph(SearchState)
graph.add_node("search", search_node)
graph.add_node("summarize", summarize_node)

graph.set_entry_point("search")
graph.add_edge("search", "summarize")
graph.add_edge("summarize", END)

# Optional in-memory checkpointer
memory = MemorySaver()
app = graph.compile(checkpointer=memory)

print("Graph ready.")


In [ ]:

def run_query(query: str) -> SearchState:
    initial: SearchState = {"query": query}
    final_state = app.invoke(initial)
    return final_state

# Example
example_query = "LangGraph documentation and examples for building search tools with DuckDuckGo"
final = run_query(example_query)
final.keys(), print(final.get("answer_markdown", ""))


In [ ]:

# Run multiple queries without reloading the kernel.
# Stop with KeyboardInterrupt or by not entering a query.
try:
    while True:
        q = input("\nEnter a query (or press Enter to stop): ").strip()
        if not q:
            break
        st = run_query(q)
        print("\n" + st.get("answer_markdown", ""))
except KeyboardInterrupt:
    pass
